# Using fastdb_client

This example is for running in Perlmutter.  You may be able to easily convert it for somewhere else.

## What is fastdb_client

It's a python program useful for connecting to instances of FASTDB.  At some level, it's just a wrapper around python `requests`, but it handles some other stuff you wouldn't want to handle manually.  For example, logging into FASTDB is not just a matter of sending a password over the net, but is a complicated handshaking of public key cryptography (created by somebody trying to be paranoid to make sure the password itself never needed to be sent over the network from your machine to the server machine).  The fastdb_client takes care of all of that for you.  It has the ability to do some format parsing of responses from the web server.  Finally, it has a built-in "sleep and retry" cycle, where if the connection to the FASTDB server fails for some reason (which does happen--- this is the Internet, after all), it will sleep a few seconds and retry, repeating that (by default) five times before giving up for good.  This can greatly increase the reliability of scripts that use the fastdb_client, saving you from having to implement your own error catching and retry loop.

## Getting an account

To use FASTDB, you need an account on the instance of the FASTDB web server you want to interact with.  As of this writing, the only one that exists is `https://fastdb-rknop-dev.lbl.gov`, and as its name suggests, it's a development server that may be down at any time, or that may have unexpected changes of contents and state at any time.  To get an account, ask Rob Knop (via Discovery Alliance Slack) for an account; I will need the username you want for the account, the name you want displayed when you are logged in, and the email address associated with the account.  Once I tell you the account is created, go to the webserver in your browser, and click on "Request Password Reset".  Put in your username, and click "Email Password Resent Link".  Shortly thereafter, in your email you should find a message with a link to click on to reset your password.  (When your account is first created, it does not have a password; you can not log into it until you set one.)

Make sure to choose a <a href="http://rknop.net/password.html">good password</a>.  FASTDB is currently not using any TFA, and I'm really hoping we won't feel the need to go that way, as that will make a lot of life a lot more complicated.

### Setting up a convenience initialization file

You can always just use the url, username, and password every time you connect to fastdb.  However, it will probably make your life easier if you create a file `.fastdb.ini` (notice the period at the beginning) in your home directory.  This file will have contents like:

```
[rknop-dev]
url = https://fastdb-rknop-dev.lbl.gov
username = <your username>
password = <your password>

[production]
url = https://desc-fastdb.lbl.gov
username = <your username>
password = <your password>
```

Each block of lines starting with a line in brackets represents one FASTDB server you might want to connect to.  The name in brackets can be whatever you want; it will be wha tyou use in order to specify the server you want to conenct to.  Below each bracketed line, you need three lines, similar to the example above, which specify the URL of the web server, your username on that server, and your password on that server.  (As of this writing you do not want to include the `[production]` block because that server does not exist.)

**Important**: the fastdb client will refuse to use this `.ini` file if it's permissions aren't sufficiently paranoid.  After you've created the file, run
```
cd
chmod go -rwx .fastdb.ini
```
to make sure that only you can read it.

There are actually a number of other configuration options you can specify here, but for now they're not that important.  Talk to Rob, or read the `fastdb_client.py` source code if you're interested.

## Setting up your environment

Hopefully, before long, the necessary python libraries will be in the desc `td_env` environment.  The FASTDB client requires that `pycryptodome` be installed (which, if you're building your own environment, is a simple matter of `pip install pycryptodome`), but as of this writing it's not in `td_env`.

### Running from the command line

If you're running from the command line, you can get a minimal environment that has what you need to run `fastdb` by running
```
source /global/cfs/cdirs/lsst/groups/TD/SOFTWARE/fastdb_deployment/fastdb_client_venv/bin/activate
```
Then, later, just run `deactivate` to leave the environment.

### Running from jupyter

At some point in the future, you will be able to just the `desc-td` kernel.  But, as of this writing, that kernel doesn't have the necessary `pycryptodome` libraries.

To set yourself up, on perlmutter, run
```
cd
mkdir -p .local/share/jupyter/kernels/fastdb-client
cd .local/share/jupyter/kernels/fastdb-client
```
once you're in that directory (which the last `cd` command did), create a text file `kernel.json` with the following contents:
```
{
  "argv": [
    "/global/cfs/cdirs/desc-td/SOFTWARE/fastdb_deployment/fastdb_client_venv/fastdb_client-env.sh",
    "-f",
    "{connection_file}"
  ],
  "display_name": "fastdb-client-env",
  "language": "python"
}
```

Thereafter, if you log into Perlmutter's Jupyter hub, you should be able to start a notebook that uses the `fastdb-client-env` kernel.  This example script was run within that kernel.

In [1]:
# The FastDB client can be found on Perlmutter in the directory
#   /global/cfs/cdirs/lsst/groups/TD/SOFTWARE/fastdb_deployment/fastdb_client
# (The client there is actually a symbolic link to something else underneath
# fastdb_deployment.  If you know what you're doing, you may want to use a
# version of the client other than the one there, but by default that's the
# right one to use.)  Put this directory in your PYTHONPATH so it will be found.
import sys
sys.path.insert( 0, '/global/cfs/cdirs/lsst/groups/TD/SOFTWARE/fastdb_deployment/fastdb_client' )

#...and import!
from fastdb_client import FASTDBClient


In [2]:
# Create your connection to fastdb.  This will log you in.
#
# There are two ways to do this.
# You can always just specify the url, username, and password, with 
#   something like:
#      fdb = FASTDBClient( 'https://fastdb-rknop-dev.lbl.gov', 'rknop', 'not_really_robs_password' )
#   where the three arguments are url, username, and password.  BE CAREFUL PUTTING PASSWORDS IN SCRIPTS,
#   and NEVER commit a script that has a password in it to a git archive anywhere!  If you do, immmediately
#   change your password.
# The other alternative is just to use the name you set up in your ~/.fastdb.ini file (described above).
#   that's what I do here.

fdb = FASTDBClient( "rknop-dev" )

## WEB API Endpoints

(Nothing currently exists.  Watch this space.)

## Direct SQL Queries

You can make make direct read-only queries to the PostgreSQL database behind the FASDTB webserver.  You construct queries and substitution dictionaries just like you'd send to `psycopg2`'s `cursor.execute` method.  (Use the `%(name)s` format for substitutions in your query, *not* the simple `%s` format where you'd pass a list as the second argument of `cursor.execute.)

There are two interfaces.  The "short" query interface is for queries that will complete in less than five minutes.  (This is actually not a database restriction, but a restriction of the web proxy in front of the FASTDB web server; it will time any connection out after five minutes.)  The "long" query interface is for longer queries.  It normally has a cycle of submitting a query, repeatingly checking to see if it's done, and then asking for the results.  However, the fastdb_client has a function that wraps all of that into one call that's as simple to use as the short SQL query.

For both interfaces, you can either pass a single query, or a list of queries.  If you pass a list of queries, you will only get the results from the *last* query.  If you want the results of multiple queries, send them each in individually.  The purpose of allowing multiple queries is so that you can play games with temporary tables and such while doing a complicated query.  If you past a list of queries, then you must also pass a list of substitution dictionaries that has the same length as your query list.

Using either interface requires you to understand the structures of the tables.  In particular, you need to undertand about `processing_version` and `snapshot`.  If you do not, and just look at columns in the `diaobject` and `diasource` tables, there is an excellent chance you will not get what you think you are getting.  See ROB PUT IN THE PLACE WHERE YOU DOCUMENT THE DATABASE SCHEMA AND STRUCTURE.

In [3]:
# The short query interface is for queries that take less than 5 minutes.
#
# If all is well, you get back either a list of dictionaries, or a dictionary of
# column→list, depending on how you call the short query interface.

# A quick example: let's get the 10 objects with the lowest diaobjectid.  (This isn't something that
# would usually be useful, but it's an easy example.)

res = fdb.submit_short_sql_query( "SELECT * FROM diaobject ORDER BY diaobjectid LIMIT 10" )
print( f"Got back a {type(res)} of length {len(res)}" )
print( f"The keys in the first element of the resonse are: {res[0].keys()}" )

Got back a <class 'list'> of length 10
The keys in the first element of the resonse are: dict_keys(['diaobjectid', 'processing_version', 'radecmjdtai', 'validitystart', 'validityend', 'ra', 'raerr', 'dec', 'decerr', 'ra_dec_cov', 'nearbyextobj1', 'nearbyextobj1id', 'nearbyextobj1sep', 'nearbyextobj2', 'nearbyextobj2id', 'nearbyextobj2sep', 'nearbyextobj3', 'nearbyextobj3id', 'nearbyextobj3sep', 'nearbylowzgal', 'nearbylowzgalsep', 'parallax', 'parallaxerr', 'pmra', 'pmraerr', 'pmra_parallax_cov', 'pmdec', 'pmdecerr', 'pmdec_parallax_cov', 'pm_ra_dec_cov'])


In [4]:
# While a list of dictionaries is sometimes convenient, it's also a wasteful format,
# as the JSON that's passed from server to client repeats all of the column names every
# single row.  You can also ask for a dictionary of columns to lists of values.
# Do this by passing return_format=1 to submit_short_sql_query.

res = fdb.submit_short_sql_query( "SELECT * FROM diaobject ORDER BY diaobjectid LIMIT 10", return_format=1 )
print( f"Got back a {type(res)} with {len(res.keys())} keys; res['diaobjectid'] has length {len(res['diaobjectid'])}" )
print( f"The diaobjectids returned are {', '.join([ str(i) for i in res['diaobjectid'] ])}" )

Got back a <class 'dict'> with 30 keys; res['diaobjectid'] has length 10
The diaobjectids returned are 1000052, 1000065, 1000111, 1000154, 1000155, 1000168, 1000283, 1000294, 1000296, 1000344


In [6]:
# You can pass variables when constructing your query. The query below is going to ask for all objects within
#   4 arcminutes of (RA, Dec) = (228°, 5.°) 
# The q3c_radial_query function is from the q3c postgres extension:
#   https://github.com/segasai/q3c
# A number of the tables have q3c indices defined for faster spatial querying.

query = "SELECT * FROM diaobject WHERE q3c_radial_query(ra,dec,%(ra)s,%(dec)s, %(rad)s)"
subs = { 'ra': 228., 'dec': 5., 'rad': 240./3600. }
res = fdb.submit_short_sql_query( query, subs, return_format=0 )
print( f"{len(res)} objects within {subs['rad']*60.:.2f} arcmin of ({subs['ra']:.5f}, {subs['dec']:.5f})" )

5 objects within 4.00 arcmin of (228.00000, 5.00000)
